In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [3]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from catboost import CatBoostClassifier

C:\Users\User\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [5]:
df = pd.read_csv('protine_cksaagp.csv')
df.head()

,0,0.1,0.1.1,0.2,0.3,0.1.2,0.4,0.5,0.6,0.7,...,0.48,0.49,0.50,0.51,0.058823529.4,0.176470588.1,0.058823529.5,0.058823529.6,0.52,0.53
0,0,0.10,0.05,0.00,0.1,0.10,0.10,0.0,0.0,0.00,...,0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.000000,0.000000,0.000000
1,0,0.20,0.05,0.10,0.0,0.05,0.05,0.0,0.0,0.00,...,0.058824,0.058824,0.058824,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000
2,0,0.25,0.05,0.15,0.0,0.05,0.00,0.0,0.0,0.05,...,0.000000,0.000000,0.000000,0.000000,0.058824,0.117647,0.000000,0.058824,0.000000,0.000000
3,0,0.15,0.00,0.15,0.1,0.05,0.00,0.0,0.0,0.00,...,0.117647,0.000000,0.058824,0.000000,0.058824,0.058824,0.000000,0.058824,0.058824,0.000000
4,0,0.15,0.05,0.05,0.1,0.10,0.00,0.0,0.0,0.00,...,0.000000,0.000000,0.000000,0.058824,0.117647,0.176471,0.000000,0.000000,0.000000,0.058824


In [6]:
y = df['0']
x = df.drop('0', axis=1) 

In [7]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

In [8]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.611836  0.006904   
1  XGBClassifier(base_score=None, booster=None, c...  0.542628 -0.049974   
2      LGBMClassifier(max_depth=10, random_state=50)  0.541625 -0.046382   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.537613 -0.014231   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.592778  0.005966   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.004140   0.387097  0.064343  0.110345     0.939103     0.064343  
1 -0.047834   0.331984  0.219839  0.264516     0.735577     0.219839  
2 -0.044692   0.335938  0.230563  0.273450     0.727564     0.230563  
3 -0.014148   0.364198  0.316354  0.338594     0.669872     0.316354  
4  0.004887   0.381295  0.142091  0.207031     0.862179     0.142091  


In [9]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain, ytrain)

In [10]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.723558  0.447903   
1  XGBClassifier(base_score=None, booster=None, c...  0.710737  0.422575   
2      LGBMClassifier(max_depth=10, random_state=50)  0.707532  0.417196   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.689904  0.385738   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.568109  0.136750   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.447115   0.711044  0.753205  0.731518     0.693910     0.753205  
1  0.421474   0.696562  0.746795  0.720804     0.674679     0.746795  
2  0.415064   0.688501  0.758013  0.721587     0.657051     0.758013  
3  0.379808   0.661664  0.777244  0.714812     0.602564     0.777244  
4  0.136218   0.562592  0.612179  0.586339     0.524038     0.612179  
